# Introduction


The efficient scheduling of classes in Higher Education Institutions (HEIs) is crucial for ensuring optimal resource utilization and a seamless learning experience. However, manual scheduling processes are prone to errors and often fail to account for complex interdependencies between factors such as teacher availability, classroom capacities, and course requirements. To address these challenges, this project focuses on developing an automated scheduling system using advanced computational tools. By leveraging optimization techniques and prescriptive analysis, the system aims to generate conflict-free schedules that meet all established criteria, fostering a harmonized educational environment and enhancing the overall efficiency of HEI operations.

# Main Objectives

1. The main objective is to create an automated system for generating schedules in an HEI.<br>
2. The system should optimize the allocation of resources such as physical spaces and teaching schedules to meet the needs of students and faculty.<br>
3. It should aim to avoid conflicts and unwanted overlaps in schedules.<br>
4. The solution should employ optimization techniques and prescriptive analysis to ensure the harmonization and effectiveness of schedules.<br>



# Instalations

Installed Pyomo using the pip command. Faced difficulties installing the GLPK library, only managing to install it using Anaconda. That's why I had to specify the directory in the solver.</br>
<i>pip install pyomo</I>
<i>conda install -c conda-forge glpk</i>

# Problem Definition

In formulating this problem, I established some basic rules. The class schedule will span from 09:00 AM to 6:00 PM, Monday through Friday, thereby simulating a typical class schedule at IPCA. With these constraints, we have:

No more than 8 hours of classes per day;</br>
Students must have at least one morning or afternoon free.</br>
This translation and revision are prepared for presentation to a teacher as part of a project.

# Importing necessary libraries

Pyomo is self explanatorie, but I'm also using a json library because I'm importing all my data from json files.

In [1]:
# Import necessary libraries and modules
from pyomo.environ import *
import json

Function that loads my files and respective variables that will hold the data.

In [2]:
# Function to load JSON data from a file
def load_json_data(filename):
    """Loads data from a JSON file and returns it."""
    with open(filename, 'r') as file:
        return json.load(file)

# Load data from JSON files
classroom_data = load_json_data('Data/classrooms.json')
time_slots_data = load_json_data('Data/time_slots.json')
lessons_data = load_json_data('Data/lessons.json')
teachers_data = load_json_data('Data/teachers.json')
cohorts_data = load_json_data('Data/cohorts.json')

# Model Creation

I've created a Pyomo ConcreteModel to formulate my optimization problem. Then, I've defined sets for teachers, lessons, days, and time slots based on the given data. I've also included classrooms as a set.

For decision variables, I've defined classroom_assignment to represent whether a lesson is assigned to a specific classroom at a certain day and time slot. Additionally, I've defined schedule variables to indicate whether a teacher is scheduled to teach a lesson on a particular day and time slot. Both variables are binary decision variables.

In [3]:
# Initialize the optimization model
model = ConcreteModel()

# Process classroom and time slot data for use in the model
classrooms_data = {classroom['code']: classroom['seats'] for classroom in classroom_data['classrooms']}
classrooms = set(classrooms_data.keys())
time_slots = {slot['id'] for slot in time_slots_data['time_slots']}
classroom_names = {classroom['code']: classroom['name'] for classroom in classroom_data['classrooms']}
time_slot_periods = {slot['id']: slot['period'] for slot in time_slots_data['time_slots']}

# Process new data structures: lessons, teachers, and cohorts
lessons = {lesson['code']: lesson for lesson in lessons_data['lessons']}
teachers = {teacher['name']: teacher for teacher in teachers_data['teachers']}
cohorts = {cohort['name']: cohort for cohort in cohorts_data['cohorts']}

# Extend model sets to include lessons, teachers, and cohorts
model.lessons = Set(initialize=lessons.keys())
model.teachers = Set(initialize=teachers.keys())
model.cohorts = Set(initialize=cohorts.keys())

# Assuming each time slot represents 2 hours
time_slot_duration = 2

# Calculate the number of required time slots for each lesson
required_time_slots = {lesson_code: lesson['hours'] // time_slot_duration 
                       for lesson_code, lesson in lessons.items()}

# Define decision variables for scheduling
model.schedule = Var(model.lessons, classrooms, time_slots, within=Binary)

# Define parameters and mappings
required_time_slots = {lesson['code']: lesson['hours'] // 2 for lesson in lessons_data['lessons']}  # Assuming 2 hours per time slot
lessons_to_cohorts = {lesson['code']: [] for lesson in lessons_data['lessons']}

# Mapping of cohort names to the lessons they are taking
cohort_lessons_mapping = {cohort: set() for cohort in cohorts}
for lesson_code, cohorts_list in lessons_to_cohorts.items():
    for cohort_name in cohorts_list:
        cohort_lessons_mapping[cohort_name].add(lesson_code)

# Constraints

Next I will define my constraints, to ensure the scheduling optimization problem adheres to specific rules and preferences.

In [4]:
# Constraint: Each lesson must be scheduled at least once
def lesson_schedule_rule(model, l):
    return sum(model.schedule[l, r, t] for r in classrooms for t in time_slots) >= 1
model.lesson_schedule = Constraint(model.lessons, rule=lesson_schedule_rule)

# Constraint: No more than one lesson in each classroom at each time slot
def unique_lesson_schedule_constraint(model, r, t):
    return sum(model.schedule[l, r, t] for l in model.lessons) <= 1
model.unique_lesson_schedule = Constraint(classrooms, time_slots, rule=unique_lesson_schedule_constraint)

# Constraint: Lessons must be scheduled for their required number of time slots
def lesson_time_requirement_rule(model, l):
    return sum(model.schedule[l, r, t] for r in classrooms for t in time_slots) == required_time_slots[l]
model.lesson_time_requirement = Constraint(model.lessons, rule=lesson_time_requirement_rule)

# Constraint: Lessons must be scheduled in the correct classroom (if specific)
def correct_classroom_constraint(model, l, r, t):
    lesson_classroom = lessons[l]['classroom']
    if lesson_classroom != "any":
        return model.schedule[l, r, t] == 0 if r != lesson_classroom else Constraint.Skip
    return Constraint.Skip
model.correct_classroom_constraint = Constraint(model.lessons, classrooms, time_slots, rule=correct_classroom_constraint)

# Constraint: Ensure no cohort has overlapping lessons
def cohort_overlap_rule(model, c, t):
    lessons_for_cohort = cohort_lessons_mapping[c]
    if not lessons_for_cohort:
        return Constraint.Skip
    scheduled_lessons_sum = sum(model.schedule[l, r, t] for l in lessons_for_cohort for r in classrooms)
    if scheduled_lessons_sum.is_fixed():
        return Constraint.Feasible if scheduled_lessons_sum() <= 1 else Constraint.Infeasible
    else:
        return scheduled_lessons_sum <= 1
model.cohort_overlap = Constraint(model.cohorts, time_slots, rule=cohort_overlap_rule)

# Creating solver

Here, I'm solving the optimization model using the GLPK (GNU Linear Programming Kit) solver. I've specified the solver to be used as 'glpk' and provided the path to the GLPSOL executable file. This command instructs Pyomo to utilize the GLPK solver to find the optimal solution for the defined mathematical model. The results of the optimization process are stored in the results variable.

# Displaying the results

In [8]:
# Solve the model
solver = SolverFactory('glpk', executable=r'C:\ProgramData\anaconda3\Library\bin\glpsol.exe')
results = solver.solve(model)

if str(results.solver.termination_condition) == "optimal":
    print("Solver terminated with optimal solution.")
    
    # Prepare and display the scheduling results
    lessons_to_cohorts = {}
    for cohort in cohorts_data['cohorts']:
        for lesson_code in cohort['lessons']:
            if lesson_code not in lessons_to_cohorts:
                lessons_to_cohorts[lesson_code] = []
            lessons_to_cohorts[lesson_code].append(cohort['name'])

    cohort_schedules = {cohort['name']: {} for cohort in cohorts_data['cohorts']}
    for l in model.lessons:
        for r in classrooms:
            for t in time_slots:
                if model.schedule[l, r, t].value == 1:
                    lesson_name = lessons[l]['name']
                    classroom_name = classroom_names[r]
                    time_slot_period = time_slot_periods[t]
                    for cohort_name in lessons_to_cohorts.get(l, []):
                        if t not in cohort_schedules[cohort_name]:
                            cohort_schedules[cohort_name][t] = []
                        cohort_schedules[cohort_name][t].append((lesson_name, classroom_name, time_slot_period))

    # Display the formatted schedule for each cohort
    for cohort_name, schedule in cohort_schedules.items():
        print(f"Cohort: {cohort_name}")
        for time_slot, lessons in sorted(schedule.items(), key=lambda x: x[0]):
            for lesson_detail in lessons:
                lesson_name, classroom_name, time_slot_period = lesson_detail
                print(f"{time_slot_period}: {lesson_name} in Classroom: {classroom_name}")
        print("\n" + "-"*40 + "\n")

else:
    print("Solver terminated with non-optimal solution.")

Solver terminated with optimal solution.
Cohort: Future Innovators
Monday Morning 09-11: Software Development Lab in Classroom: IT Lab
Monday Morning 09-11: Computer Networks in Classroom: Advanced IT Lab
Wednesday Morning 11-13: Software Development Lab in Classroom: IT Lab
Wednesday Morning 11-13: Computer Networks in Classroom: Advanced IT Lab
Wednesday Afternoon 14-16: Software Development Lab in Classroom: IT Lab
Wednesday Afternoon 16-18: Foundational Mathematics in Classroom: Biology Lab
Wednesday Afternoon 16-18: Data Science with Python in Classroom: IT Lab
Thursday Morning 09-11: Data Science with Python in Classroom: IT Lab
Thursday Morning 11-13: Foundational Mathematics in Classroom: Biology Lab
Thursday Morning 11-13: Computer Networks in Classroom: Advanced IT Lab
Thursday Morning 11-13: Data Science with Python in Classroom: IT Lab
Thursday Afternoon 14-16: Foundational Mathematics in Classroom: Biology Lab
Thursday Afternoon 14-16: Data Science with Python in Classroom

In [10]:
def write_results_to_file(results, filename):
    """
    Writes scheduling results to a text file.

    Parameters:
    results (str): The text containing the scheduling results.
    filename (str): The name of the file where results should be written.

    Returns:
    None
    """
    # Open the file in write mode or create it if it doesn't exist
    with open(filename, 'w') as file:
        # Write the results to the file
        file.write(results)
    print(f"Results successfully written to {filename}")

# Example usage
results_text = """
Cohort: Future Innovators
Monday Morning 09-11: Software Development Lab in Classroom: IT Lab
...
----------------------------------------
Cohort: Art and Design Pioneers
Monday Morning 09-11: Digital Media Design in Classroom: Design Lab
...
"""

# Assuming you want to write to 'schedule_results.txt'
filename = 'schedule_results.txt'
write_results_to_file(results, filename)


TypeError: write() argument must be str, not SolverResults